In [ ]:
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import json
import torch
import numpy as np
import evaluate

ModuleNotFoundError: No module named 'evaluate'

In [24]:
with open('corpus_complete.json') as corpusfile:
    corpus = json.load(corpusfile)

In [ ]:
text = corpus[0]["products"][0]["languages"]["en"][2]["paragraphText"]
text = text.replace(" *", ".")
text = text.split(".")
print(text)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt") # json input_ids, token_type_ids, attention_mask
print(tokenized)
tokenizer.decode(tokenized["input_ids"][0])

In [ ]:
# model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')
model = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
model.save_pretrained("./model_test")